In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=6d1eaebd586118c15445755d4853c53ed791df17f5f7cca22306d3762baef788
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
!wget http://rtw.ml.cmu.edu/resources/results/08m/NELL.08m.1115.esv.csv.gz

--2023-09-24 17:14:05--  http://rtw.ml.cmu.edu/resources/results/08m/NELL.08m.1115.esv.csv.gz
Resolving rtw.ml.cmu.edu (rtw.ml.cmu.edu)... 128.2.220.128
Connecting to rtw.ml.cmu.edu (rtw.ml.cmu.edu)|128.2.220.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929107246 (886M) [application/x-gzip]
Saving to: ‘NELL.08m.1115.esv.csv.gz’

NELL.08m.1115.esv.c 100%[===================>] 886.07M  74.4MB/s    in 13s     

2023-09-24 17:14:18 (69.3 MB/s) - ‘NELL.08m.1115.esv.csv.gz’ saved [929107246/929107246]



In [4]:
!gunzip /content/NELL.08m.1115.esv.csv.gz

In [5]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [8]:
path = "/content/NELL.08m.1115.esv.csv"

In [21]:
df = spark.read \
.option("delimiter", "\t") \
.csv(path)

In [26]:
df.show(2)

+--------------------+---------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|            _c1|                 _c2|                 _c3|               _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c9|                _c10|                _c11|                _c12|
+--------------------+---------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              Entity|       Relation|               Value|Iteration of Prom...|       Probability|              Source|Entity literalStr...|Value literalStrings|Best Entity liter...|Best Value li

In [29]:
# Get the relevant columns
df = df[['_c0', '_c1', '_c2']]

In [31]:
# Take the first 50 rows
df = df.limit(50)

# Convert to Pandas DataFrame for easier processing
pandas_df = df.toPandas()

# Create an empty knowledge graph
knowledge_graph = {}

# Iterate through the data and populate the graph
for _, row in pandas_df.iterrows():
    entity1 = row['_c0']
    relation = row['_c1']
    entity2 = row['_c2']

    if entity1 not in knowledge_graph:
        knowledge_graph[entity1] = {}
    if relation not in knowledge_graph[entity1]:
        knowledge_graph[entity1][relation] = []

    knowledge_graph[entity1][relation].append(entity2)

# Print the resulting knowledge graph
for entity, relations in knowledge_graph.items():
    for relation, values in relations.items():
        print(f"{entity} --{relation}--> {values}")

Entity --Relation--> ['Value']
concept:biotechcompany:aspect_medical_systems --generalizations--> ['concept:biotechcompany']
concept:company:limited_brands --concept:companyceo--> ['concept:ceo:leslie_wexner']
concept:company:limited_brands --generalizations--> ['concept:retailstore', 'concept:company']
concept:biotechcompany:calavo_growers --generalizations--> ['concept:biotechcompany']
concept:biotechcompany:playboy_enterprises_inc --generalizations--> ['concept:biotechcompany']
concept:biotechcompany:b_c_s_b_bancorp_inc --generalizations--> ['concept:biotechcompany']
concept:biotechcompany:boeing_integrated_defense_systems --generalizations--> ['concept:biotechcompany']
concept:company:mahindra___mahindra_ltd_ --generalizations--> ['concept:company']
concept:biotechcompany:marsam_pharmaceuticals --generalizations--> ['concept:biotechcompany']
concept:biotechcompany:sifco_industries_inc --generalizations--> ['concept:biotechcompany']
concept:biotechcompany:medecins_sans_frontieres --